### Load Configurations

In [3]:
import os
import yaml

with open('../config.yml', 'r') as f:
    conf = yaml.safe_load(f)
    
API_KEY = conf['openai-api-key']
os.environ["OPENAI_API_KEY"] = API_KEY

### Default Usage

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)
response = llm("Tell me a joke")
print(response)



Why did the chicken cross the road?

To get to the other side!


In [3]:
response

'\n\nWhy did the chicken cross the road?\n\nTo get to the other side!'

### Default Prompting

## What is a prompt template?

A prompt template refers to a reproducible way to generate a prompt. It contains a text string (“the template”), that can take in a set of parameters from the end user and generate a prompt.
The prompt template may contain:

- instructions to the language model,
- a set of few shot examples to help the language model generate a better response,
- a question to the language model.

The following code snippet contains an example of a prompt template:

In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
prompt

PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='What is a good name for a company that makes {product}?', template_format='f-string', validate_template=True)

- `input_variables`: 프롬프팅 템플릿에 들어가는 변수 명을 의미함
- `template`: 프로프팅 템플릿 텍스트 정보를 의미함

In [5]:
template = """
I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt

PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='\nI want you to act as a naming consultant for new companies.\n\nHere are some examples of good company names:\n\n- search engine, Google\n- social media, Facebook\n- video sharing, YouTube\n\nThe name should be short, catchy and easy to remember.\n\nWhat is a good name for a company that makes {product}?\n', template_format='f-string', validate_template=True)

In [6]:
# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
no_input_prompt.format()
# -> "Tell me a joke."

# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a {adjective} joke.")
one_input_prompt.format(adjective="funny")
# -> "Tell me a funny joke."

# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"], 
    template="Tell me a {adjective} joke about {content}."
)
multiple_input_prompt.format(adjective="funny", content="chickens")
# -> "Tell me a funny joke about chickens."

'Tell me a funny joke about chickens.'

## Output Parser

In [2]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [4]:
model_name = "text-ada-001"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [5]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

In [6]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [34]:
import pprint

format_instruct = parser.get_format_instructions()
pprint.pprint(format_instruct)

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"setup": {"title": "Setup", "description": "question to set '
 'up a joke", "type": "string"}, "punchline": {"title": "Punchline", '
 '"description": "answer to resolve the joke", "type": "string"}}, "required": '
 '["setup", "punchline"]}\n'
 '```')


In [35]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['query'], output_parser=None, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'}, template='Answer the user query.\n{format_instructions}\n{query}\n', template_format='f-string', validate_template=True)

In [36]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)
_input

StringPromptValue(text='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke.\n')

In [25]:
pprint.pprint(_input.to_string())

('Answer the user query.\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"setup": {"title": "Setup", "description": "question to set '
 'up a joke", "type": "string"}, "punchline": {"title": "Punchline", '
 '"description": "answer to resolve the joke", "type": "string"}}, "required": '
 '["setup", "punchline"]}\n'
 '```\n'
 'Tell me a joke.\n')


In [37]:
output = model(_input.to_string())

In [38]:
output

'\nThe output should be a JSON instance that conforms to the JSON schema below.'

In [40]:
# parser.parse(output) # error but why?